Hi!

So we begin with importing some things we need.

I'm importing the file vgg16.py, a well documented file as to import and use the vgg16.tfmodel

Resources used :

In [0]:
from __future__ import print_function
import numpy as np
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt

In [0]:
import os
work_dir = "/content/"
os.chdir(work_dir)

In [0]:
import vgg16
vgg16.maybe_download()

First, we load the style and content images.

As I made this in Google Colab, I did upload *my* **style_image** and **content_image** into it. If you want to use any other image, just provide the links to them, or you might even use Google Drive.



Subsequently, the images are converted to a matrix of entries with type float.

In [0]:
style_image= Image.open('style_image.jpg')
#style_image = Image.open('your_link_here')

content_image = Image.open('content_image.png')
#style_image = Image.open('your_link_here')

I used 512x512 images. Resize if you want to.

In [0]:
"""
  def resize(image, size):
  
"""

In [0]:
style_image = np.float32(style_image) 
content_image = np.float32(content_image)        #conversion to array

I won't go exactly with the paper. I'd make two loss functions for now, 

*content_loss*, and *style_loss*...

In [0]:
def content_image_loss(session, model, content_image, layer_ids):
  
  feed_dict = model.create_feed_dict(image = content_image)
  layers = model.get_layer_tensors(layer_ids = layer_ids)
  
  value_in_layers = session.run(layers, feed_dict = feed_dict)
  
  with model.graph.as_default():
    iterator = zip(values_in_layers, layers)
  
    content_loss = []
  
    for value, layer in iterator:
      loss = tf.reduce_mean(tf.square(value - layer))        #mean squared error
      content_loss.append(loss)
    
  loss = tf.reduce_mean(content_loss)                      #total loss is the average of all layer losses
  return(loss)

In [0]:
#Consider the layer has 5 channels
#num_channels = 5
#we need the product of the activation_values of all points through all these channels. 
#thus, it is reshaped to a [(suitable_size)xnum_channels] matrix 

def gram_matrix(feature):
  shape = feature.get_shape()
  num_channels = int(shape[3])
  
  matrix = tf.reshape(feature, shape = [-1,num_channels])
  gram = tf.matmul(tf.transpose(matrix), matrix)
  
  return(gram)

In [0]:
def style_image_loss(session, model, style_image, layer_ids):
  
  feed_dict = model.create_feed_dict(image = style_image)
  layers = model.get_layer_tensors(layer_ids = layer_ids)

<class 'tensorflow.python.framework.ops.Tensor'>
